# Data preparation and cleaning

In [140]:
# Imports
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np
import matplotlib.pyplot as plt

## Detecting problems

In [141]:
dft = pd.read_csv("data/titanic_processed.csv")
dft.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1063.000000,1063.000000,1063.000000,855.000000,1063.000000,1063.000000,1063.000000
mean,447.238946,0.379116,2.301035,31.216480,0.512700,0.382879,32.547749
std,260.770475,0.485395,0.837936,18.641281,1.083856,0.807899,48.902782
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,224.500000,0.000000,2.000000,21.000000,0.000000,0.000000,7.910400
50%,444.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.458300
75%,673.500000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,891.000000,1.000000,3.000000,130.000000,8.000000,6.000000,512.329200


### Empty values

The first and most obvious thing to check is whether there are empty values in the data.


In [142]:
for col in dft.columns:
    print(f'Column {col: <12} has {dft[col].isna().sum(): 4d} NaN values')


Column PassengerId  has    0 NaN values
Column Survived     has    0 NaN values
Column Pclass       has    0 NaN values
Column Name         has    0 NaN values
Column Sex          has    0 NaN values
Column Age          has  208 NaN values
Column SibSp        has    0 NaN values
Column Parch        has    0 NaN values
Column Ticket       has    0 NaN values
Column Fare         has    0 NaN values
Column Cabin        has  816 NaN values
Column Embarked     has    3 NaN values


### Duplicates

Records have an unpleasant tendency to be duplicated. Luckily Pandas can help detect this.


In [143]:
print(f'Number of duplicates is {dft.duplicated( keep='first').sum()}')

Number of duplicates is 172


### Nonsensical values

Let's check each column to see if the values make sense.

For numerical columns we do this by checking the range of values while for text columns we list the unique values.

This data set has one column for which this is impossible: we likely have no way of determining whether a value in the "Name" column is plausible or not. The same is true for the "Ticket" column which holds the ticket number and for the "Cabin" column which holds the cabin number. It's probably also not useful to check what's in the "PassengerId" column.

In [144]:
columns_to_skip = ['PassengerId', 'Name', 'Ticket', 'Cabin']
for col in dft.columns:
    if col in columns_to_skip:
        continue
    if is_numeric_dtype(dft[col]):
        print(f'Column "{col}" has range {dft[col].min()} - {dft[col].max()}')
    else:
        print(f'Column {col} has values {dft[col].unique()} ')

Column "Survived" has range 0 - 1
Column "Pclass" has range 1 - 3
Column Sex has values ['male' 'female' 'caprio' 'fmale'] 
Column "Age" has range 0.42 - 130.0
Column "SibSp" has range 0 - 8
Column "Parch" has range 0 - 6
Column "Fare" has range 0.0 - 512.3292
Column Embarked has values ['S' 'C' 'Q' nan] 


## Fixing problems

### Duplicates

First of all we want to get rid of the duplicate records Pandas detected for us above.

How this works:
- `df.duplicated(keep = 'first')` returns a list of True / False values. "True" if the record is considered to be a duplicate, "False" if it's not.
- We then use this list to filter the original dataframe. Normally, applying a list of True / False values as a filter (`df[list_of_true_false]`) will return only those records for which the list has a value of "True". In this case, "True" means the record is a duplicate. What we actually want is to retain only those records for which the list has a value of False. We accomplish this by using the `~` operator (logical NOT) before the filter.

In [145]:
dft_clean = dft[~dft.duplicated( keep='first')]
# Verify there are no more duplicates:
print(f'After de-duplicating we are left with {dft_clean.duplicated(keep = 'first').sum()} duplicates')



After de-duplicating we are left with 0 duplicates


### Dealing with nonsensical values

We have found two columns with nonsensical values:
- Not all values in the "Sex" column are correct. Some of these incorrect values are obvious typos, others are plain hot garbage.
- The "Age" column has values ranging from 0.4 to 130. Even today people rarely age to more than 100 years, let alone in 1912.

#### Dealing with nonsensical values in the "Sex" column

The obvious typos in the "Sex" column are easy enough to fix.

Again we use a filter (list of True / False) for this. The filter gives "True" for all records where the "Sex" column matches the condition, false where it doesn't. We can use this to update only those values that match the condition.

In [146]:
dft_clean.loc[dft_clean['Sex'] == 'fmale', 'Sex'] = 'female'
# Verify that the typos are gone.
dft_clean['Sex'].unique()

array(['male', 'female', 'caprio'], dtype=object)

Some incorrect values are a bit more difficult to deal with. We really have no way of knowing what "caprio" means.

To come up with a solution, we need to know how many records contain this value.

In [147]:
print(f'The number of records with a nonsensical value in the "Sex" column is {len(dft_clean[dft_clean['Sex'] == 'caprio'])} (out of a total of {len(dft_clean)} records so {(len(dft_clean[dft_clean['Sex'] == 'caprio']) / len(dft_clean)) * 100 :.1f}% of the total')

The number of records with a nonsensical value in the "Sex" column is 21 (out of a total of 891 records so 2.4% of the total


Now that we know what we're dealing with, we have to make a decision. Throw them out or replace them with values that make sense? And if the latter: what does "values that make sense" mean? For numerical values we can use the mean, median or mode but for a categorical value we have to do more work.

One option is simply to assign the value that is the most common. For this we need to know the counts for each value. The `value_counts` command tells us this (`group_by` with `count` would also work).

In [148]:
dft_clean['Sex'].value_counts()


Sex
male      549
female    321
caprio     21
Name: count, dtype: int64

In this case simply assigning the majority correct value is not the best option as the difference between the most common correct value and the least common correct value is not very large.

An especially fancy way to handle this problem is to use something known as "K-Nearest Neighbor Imputation". For this technique, "similarity" is calculated between rows (by treating each row as a coördinate in a multi-dimensional space). We can then replace the incorrect values with the correct values in the nearest neighbors (hence the name of the algorithm - $K$ is simply the *number* of neighbors to consider).

Of course we will have a problem if it turns out that the incorrect records only have neighbors that are incorrect themselves.

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer
# We need to create a new data frame containing only the numeric frames because KNNImputer only works with numeric values.
# Reuse the columns_to_skip array we made above. Notice the nested list comprehension.
dft_imp = dft_clean[[c for c in dft_clean.columns if c not in columns_to_skip]].copy()

# We need to give the "Sex" column numerical values so we can use it with KNNImputer.
# Notice we use a dict to translate string values to numbers. This allows us to reuse it later.
translation = {
    'female' : 1,
    'male' : 2,
    # The sex we want to impute needs to be set to NaN so KNNImputer can fill it.
    'caprio' : np.nan,
}
for (orig, new) in translation.items():
    dft_imp.loc[dft_imp['Sex'] == orig, 'Sex'] = new



# The "Embarked" column also has non-numeric values. Since the exact value doesn't
# matter, just give it any value
emb_val = 1
for val in dft_imp['Embarked'].unique():
    dft_imp.loc[dft_imp['Embarked'] == val, 'Embarked'] = emb_val
    emb_val +=1


# Now run the imputer. Let's go with a low value for the number of neighbors to consider.
imputer = KNNImputer(n_neighbors=2)
imputed = imputer.fit_transform(dft_imp)
print(f"After imputation we have: {imputed}")

# Woops - we no longer have a data frame. We need to restore it.
imputed = pd.DataFrame(imputed, columns = dft_imp.columns)

# Let's see if it worked.
print(f"After imputation we end up with the following unique values in the 'Sex' column: {imputed['Sex'].unique()}")

# Ok. That didn't work. In some cases the imputer couldn't decide on a value so it took the average.
# Let's see how often that happened:
print(f"Number of records with a value of 1.5: {len(imputed[imputed['Sex'] == 1.5])}")
print(imputed['Sex'].value_counts())

# Let's see if increasing the number of neighbors helps:
imputer = KNNImputer(n_neighbors=10)
imputed = imputer.fit_transform(dft_imp)
imputed = pd.DataFrame(imputed, columns = dft_imp.columns)
print(f"After increasing the number of neighbors we end up with the following: \n{imputed['Sex'].value_counts()}")

# You can try other values for n_neighbors but you will find things never really improve.
# Let's decide to solve the problem by rounding to the nearest integer.
imputed['Sex'] = imputed['Sex'].round(0)

# Finally translate the imputed sex column back to strings using the translation map above.
# We need to change the data type to a string representation of an int to avoid complaints.
imputed['Sex'] = imputed['Sex'].astype(int).astype(str)
for stringval, number in translation.items():
    number_as_string = str(number)
    imputed.loc[imputed['Sex'] == number_as_string, 'Sex'] = stringval

# And, of course, update the original Sex column to the new Sex column.
# To make things easier to work with, create a new dataframe.
# Notice use of reset_index. This is necessary to make sure the original columns
# and the imputed column line up correctly.
dft_clean_sex = dft_clean[[col for col in dft_clean.columns if col != 'Sex']].reset_index()
dft_clean_sex['Sex'] = imputed.reset_index()['Sex']


After imputation we have: [[  0.       1.       2.     ...   1.      79.65     1.    ]
 [  1.       3.       1.     ...   1.      12.475    1.    ]
 [  0.       3.       2.     ...   0.       8.05     1.    ]
 ...
 [  1.       1.       1.     ...   0.     135.6333   2.    ]
 [  1.       1.       2.     ...   1.     512.3292   2.    ]
 [  0.       3.       1.     ...   4.      21.075    1.    ]]
After imputation we end up with the following unique values in the 'Sex' column: [2.  1.  1.5]
Number of records with a value of 1.5: 7
Sex
2.0    558
1.0    326
1.5      7
Name: count, dtype: int64
After increasing the number of neighbors we end up with the following: 
Sex
2.0    550
1.0    321
1.7      6
1.4      3
1.9      3
1.5      3
1.8      2
1.6      2
1.3      1
Name: count, dtype: int64


Missing at Random
Missing completely at Random
Missing not at Random